In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import csv
import sys
import pandas as pd
import numpy as np

In [7]:
path_data = '../datasets/WHO-covid19-clinicaltrials.csv'
df = pd.read_csv(path_data)
df.head(17)

,Platform,Type of candidate vaccine,Developer,Coronavirus target,Current stage of clinical evaluation/regulatory status- Coronavirus candidate,Same platform for non-Coronavirus candidates
0,Non- Replicating Viral Vector,ChAdOx1-S,University of Oxford/AstraZeneca,SARS-CoV2,Phase 3 ISRCTN89951424\nPhase2b/3\n2020-001228...,"MERS, influenza, TB, Chikungunya, Zika, MenB, ..."
1,Non- Replicating Viral Vector,Adenovirus Type 5 Vector,CanSino Biological Inc./Beijing Institute of B...,SARS-CoV2,Phase 2 ChiCTR2000031781\nPhase 1\nChiCTR20000...,Ebola
2,RNA,LNP-\nencapsulated mRNA,Moderna/NIAID,SARS-CoV2,Phase 2 NCT04405076\nPhase 1 NCT04283461,multiple candidates
3,Inactivated,Inactivated,Wuhan Institute of Biological\nProducts/Sinopharm,SARS-CoV2,Phase 1/2 ChiCTR2000031809,NaN
4,Inactivated,Inactivated,Beijing Institute of Biological Products/Sinop...,SARS-CoV2,Phase 1/2 ChiCTR2000032459,NaN
5,Inactivated,Inactivated + alum,Sinovac,SARS-CoV2,Phase 1/2 NCT04383574 NCT04352608,SARS
6,Protein Subunit,Full length recombinant SARS CoV-2\nglycoprote...,Novavax,SARS-CoV2,Phase 1/2 NCT04368988,"RSV; CCHF, HPV, VZV, EBOV"
7,RNA,3 LNP-mRNAs,BioNTech/Fosun Pharma/Pfizer,SARS-CoV2,Phase 1/2\n2020-001038-36\nNCT04368728,NaN
8,Inactivated,Inactivated,"Institute of Medical Biology\n, Chinese Academ...",SARS-CoV2,Phase 1 NCT04412538,NaN
9,DNA,DNA plasmid vaccine with electroporation,Inovio Pharmaceuticals,SARS-CoV2,Phase 1 NCT04336410,multiple candidates


In [22]:
developers = df['Developer'].tolist()
companies = []

# Function to see if organization is a university
def is_academia(organization: str):
    keywords = ['University', 'College', 'Academy']
    
    # https://stackoverflow.com/questions/8122079/python-how-to-check-a-string-for-substrings-from-a-list
    tf = any(map(organization.__contains__, keywords))
    return tf


# For each row, split to individual organizations
for developer in developers:
    organizations = developer.split('/')
    
    # For each organization, clean up formatting and add to companies list if not academia
    for organization in organizations:
        organization = organization.replace('\n', ' ')
        if not is_academia(organization):
            companies.append(organization)

print(companies)
        

['AstraZeneca', 'CanSino Biological Inc.', 'Beijing Institute of Biotechnology', 'Moderna', 'NIAID', 'Wuhan Institute of Biological Products', 'Sinopharm', 'Beijing Institute of Biological Products', 'Sinopharm', 'Sinovac', 'Novavax', 'BioNTech', 'Fosun Pharma', 'Pfizer', 'Inovio Pharmaceuticals', 'Genexine Consortium', 'Gamaleya Research Institute', 'Clover Biopharmaceuticals Inc.', 'GSK', 'Dynavax', 'Anhui Zhifei Longcom Biopharmaceutical', 'Curevac', 'Walvax Biotech.']


# Get company ticker from company name
Just using Wikipedia for the time being.

A more robust solution would probably be to use Yahoo! Finance's API

e.g. http://d.yimg.com/autoc.finance.yahoo.com/autoc?query=COMPANY_NAME_HERE&region=1&lang=en&callback=YAHOO.Finance.SymbolSuggest.ssCallback

In [50]:
# 12 American Securities Markets
am_mkts = ['NYSE', 'NASDAQ', 'AMEX', 'BSE', 'CBOE', 'CBOT', 'CME', 'CHX', 'ISE', 'MS4X', 'NSX', 'PHLX']

### Scraping Wikipedia Approach

In [67]:
import wptools

def get_stock_info(page):
    try:
        stocks = page.data['infobox']['traded_as']
        stocks = stock.replace('{', '').replace('}', '').split('<br />')

    except:
        print('Not a publicly traded company')
        stocks = 'n/a'
        
    finally:
        return stocks


def check_am_mkts(stocks):
    am_mkts = ['NYSE|', 'NASDAQ|', 'AMEX|', 'BSE|', 'CBOE|', 'CBOT|', 'CME|', 'CHX|', 'ISE|', 'MS4X|', 'NSX|', 'PHLX|']
    tf_array = []
    
    for stock in stocks:
        tf = any(map(stock.upper().__contains__, am_mkts))
        tf_array.append(tf)
        
    return tf_array
   

def get_stock_symbol(stocks):
    if stocks[0] == 'n/a':
        return stock
    
    tf_array = check_am_mkts(stocks)
    
    exchange = stocks[0].split('|')[0]
    stock = stocks[0].split('|')[1]
    
    for i, tf in enumerate(tf_array):
        if tf:
            exchange = stocks[i].split('|')[0]
            stock = stocks[i].split('|')[1]
            
    stock = stock.replace(' ', '').upper()
    
    return stock, exchange
    
# def get_location(page):
#     try:
#         location = page.data['infobox']['location']
#         print(location)

symbols = []
exchanges = []

for company in companies:
    print(f'Looking up {company}')
    
    try:
        page = wptools.page(company).get_parse()
        stocks = get_stock_info(page)
        symbol, exchange = get_stock_symbol(stocks)

        symbols.append(symbol)
        exchanges.append(exchange)
    except:
        print(f'Wiki page could not be found for {company}')
        symbols.append('n/a')
        exchanges.append('n/a')
        
    print(symbols)
    print(exchanges)



Looking up AstraZeneca


en.wikipedia.org (parse) AstraZeneca
AstraZeneca (en) data
{
  infobox: <dict(17)> name, logo, type, traded_as, predecessors, f...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:A...
  pageid: 441872
  parsetree: <str(71937)> <root><template><title>short description...
  requests: <list(1)> parse
  title: AstraZeneca
  wikibase: Q731938
  wikidata_url: https://www.wikidata.org/wiki/Q731938
  wikitext: <str(58695)> {{short description|British-Swedish pharm...
}
en.wikipedia.org (parse) CanSino Biological Inc.


['AZN']
[' nyse']
Looking up CanSino Biological Inc.


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Beijing Institute of Biotechnology


Wiki page could not be found for CanSino Biological Inc.
['AZN', 'n/a']
[' nyse', 'n/a']
Looking up Beijing Institute of Biotechnology


...

GSK (en) data
{
  pageid: 1557420
  parsetree: <str(373)> <root>'''GSK''' may refer to:*[[Galatasara...
  requests: <list(1)> parse
  title: GSK
  wikibase: Q346413
  wikidata_url: https://www.wikidata.org/wiki/Q346413
  wikitext: <str(314)> '''GSK''' may refer to:*[[Galatasaray S.K.]...
}
en.wikipedia.org (parse) Dynavax
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Anhui Zhifei Longcom Biopharmaceutical


Not a publicly traded company
Wiki page could not be found for GSK
['AZN', 'n/a', 'n/a', 'AZN', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'AZN', 'n/a', 'AZN', 'AZN', 'AZN', 'n/a', 'n/a', 'n/a', 'n/a']
[' nyse', 'n/a', 'n/a', ' nyse', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', ' nyse', 'n/a', ' nyse', ' nyse', ' nyse', 'n/a', 'n/a', 'n/a', 'n/a']
Looking up Dynavax
Wiki page could not be found for Dynavax
['AZN', 'n/a', 'n/a', 'AZN', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'AZN', 'n/a', 'AZN', 'AZN', 'AZN', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a']
[' nyse', 'n/a', 'n/a', ' nyse', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', ' nyse', 'n/a', ' nyse', ' nyse', ' nyse', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a']
Looking up Anhui Zhifei Longcom Biopharmaceutical


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Curevac


Wiki page could not be found for Anhui Zhifei Longcom Biopharmaceutical
['AZN', 'n/a', 'n/a', 'AZN', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'AZN', 'n/a', 'AZN', 'AZN', 'AZN', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a']
[' nyse', 'n/a', 'n/a', ' nyse', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', ' nyse', 'n/a', ' nyse', ' nyse', ' nyse', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a']
Looking up Curevac
Not a publicly traded company
Wiki page could not be found for Curevac
['AZN', 'n/a', 'n/a', 'AZN', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'AZN', 'n/a', 'AZN', 'AZN', 'AZN', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a']
[' nyse', 'n/a', 'n/a', ' nyse', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', ' nyse', 'n/a', ' nyse', ' nyse', ' nyse', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a']
Looking up Walvax Biotech.
Wiki page could not be found for Walvax Biotech.
['AZN', 'n/a', 'n/a', 'AZN', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'AZN', 'n/a', 'AZN', 'AZN', 'AZN', 'n/a', 'n/a', 'n/a', 'n/a'

CureVac (en) data
{
  infobox: <dict(11)> name, logo, type, founded, founders, locatio...
  iwlinks: <list(2)> https://commons.wikimedia.org/wiki/Category:C...
  pageid: 39161114
  parsetree: <str(15906)> <root><template><title>short description...
  requests: <list(1)> parse
  title: CureVac
  wikibase: Q1144473
  wikidata_url: https://www.wikidata.org/wiki/Q1144473
  wikitext: <str(12368)> {{short description|German bio-pharmaceut...
}
en.wikipedia.org (parse) Walvax Biotech.
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}


### Using Yahoo! Finance Approach

## Get market value

Market value or market cap can be pulled using the pandas_datareader module
Examples with Apple, Amazon, Tesla, and Google seen below (output is in dollars)

```python
from pandas_datareader import data
tickers = ['AAPL','AMZN','TSLA','GOOG']
data.get_quote_yahoo(tickers)['marketCap']
```

Stackoverflow link: https://stackoverflow.com/questions/54815864/downloading-a-companies-market-cap-from-yahoo